<a href="https://colab.research.google.com/github/silverioleitejr/avaliacao-python/blob/main/kpmg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
! pip install pandas pyspark fastapi nest-asyncio pyngrok uvicorn 


     |████████████████████████████████| 212.3MB 72kB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 747kB 43.4MB/s 
     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 204kB 47.4MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 10.1MB 39.6MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=2634833221c8fd333b55d3c349bfa13f8c1a39a7fd6ddbc4ccbd31fcb89c8e06
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
  Created wheel for pyngrok: filename=pyngrok-5.0.3-cp37-none-any.whl size=18979 sha256=8993668e95b1142cbbde301ea6c1c15cfb3a19e11c1d4d646d5a35dd41d432fa
  Stored in directory: /root/.cache/pip/wheels/f0/77/01/d23761888c52099d1817121dc1f28afbc1c7cbf3caac93b1ff
Successfully built pyspark pyngr

In [136]:

################################################################
# Projeto de Avaliação - Python - KPMG
#
# Analista responsavel: Silverio Leije Jr
# Data: 06/03/2021
# Consultoria: Pasqualini
################################################################
# I.	  rota que retornará o valor médio de todos os carros baseados no fabricante (coluna car_make agrupada pela média dos valores da coluna car_value);
# II.	  rota que retornará o valor médio de um fabricante de carro passado como parâmetro no request (Após o cálculo de I, pode-se filtrar apenas o valor requerido);
# III.	rota que que retornará o valor médio de todos os carros baseados nas cidades correspondentes (coluna city agrupada pela média dos valores da coluna car_value);
# IV.	  rota que retornará o valor médio dos carros de uma cidade passada como parâmetro no request (Após o cálculo de III, pode-se filtrar apenas o valor requerido);
#################################################################


#definir variais globais 
import pandas as pd
pd.options.display.float_format = '${:,.2f}'.format
path = "https://raw.githubusercontent.com/silverioleitejr/avaliacao-python/main/data/dataset.csv"
vendas_df = pd.read_csv(path)

#Bibliotecas para API
from fastapi import FastAPI 
from pydantic import BaseModel
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *

app = FastAPI()

#verificar se um valor é nulo
def isNaN(num):
    return num != num


# gerar dataframe agrupamento de valores por cidades 
def agruparMediaCidade():
  spark = SparkSession.builder.getOrCreate()

  # Ler o dataframe de vendas 
  spark_df = spark.createDataFrame(vendas_df)

  # incluir o spark df no cataloga Spark
  spark_df.createOrReplaceTempView('venda')
  agrupar_cidade_df = spark_df.groupBy("city").mean("car_value")  
  #agrupar_cidade_df = agrupar_cidade_df.select(col("avg(car_value)").alias("Valor Medio"))  

  return agrupar_cidade_df


#rota raiz
@app.get('/')
def raiz():
      return {"Mensagem" : "API de avaliação - codigo fonte escrito em python"}

### uvicorn main:app --reload
### http://127.0.0.1:8000/
### http://127.0.0.1:8000/docs

# rota para retornar media por fabricante
# item I da avaliação
# http://127.0.0.1:8000/valor-medio-todos-fabricante
@app.get('/valor-medio-todos-fabricante')
def media_todos_fabricante():
  grouped_df = vendas_df.groupby(['car_make'])
  mean_df = grouped_df.mean()
  mean_df.drop(['id'], inplace=True, axis=1)
  mean_df.drop(['car_year'], inplace=True, axis=1)

  return mean_df
  
# rota para retornar media para fabricante informado no parametro
# item II da avaliação
# http://127.0.0.1:8000/valor-medio-fabricante/A
@app.get('/valor-medio-fabricante/{fabricante}')
def media_fabricante(fabricante:str):
  agrupamento_df = agruparMediaMarca()
  filtro_query = 'car_make == ' '"' + fabricante + '"'
  filtro_df = agrupamento_df.query( filtro_query )
  media = filtro_df['car_value']
  return { "car_make" : media}

# rota para retornar media por cidade - city
# item III da avaliação
# http://127.0.0.1:8000/valor-medio-cidade
@app.get('/valor-medio-todas-cidade')
def media_todas_cidades():
  mediaCidades = agruparMediaCidade()
  return_df = mediaCidades.toPandas()
  return_df.rename(columns = {'avg(car_value)': 'Valor Médio'}, inplace = True)
  print(return_df)
  return return_df








# Testar funcao spark para agrupar cidade

In [129]:
x = agruparMediaCidade()
x.show()

media_todas_cidades()

+----+------------------+
|city|    avg(car_value)|
+----+------------------+
|C_04|47865.873239436616|
|C_07| 48582.93661971831|
|C_05| 48332.38345864662|
|C_01| 47869.25342465754|
|C_03| 47706.17605633803|
|C_06|47592.420118343194|
|C_02|  46464.6746031746|
+----+------------------+



,city,avg(car_value)
0,C_04,"$47,865.87"
1,C_07,"$48,582.94"
2,C_05,"$48,332.38"
3,C_01,"$47,869.25"
4,C_03,"$47,706.18"
5,C_06,"$47,592.42"
6,C_02,"$46,464.67"


# Executar aplicação no servidor Web

In [139]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

t=2021-03-08T13:31:27+0000 lvl=warn msg="can't bind default web address, trying alternatives" obj=web addr=127.0.0.1:4040


Public URL: http://0ef314080120.ngrok.io


INFO:     Started server process [57]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     191.188.211.138:0 - "GET / HTTP/1.1" 200 OK
INFO:     191.188.211.138:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
   city  Valor Médio
0  C_04   $47,865.87
1  C_07   $48,582.94
2  C_05   $48,332.38
3  C_01   $47,869.25
4  C_03   $47,706.18
5  C_06   $47,592.42
6  C_02   $46,464.67
INFO:     191.188.211.138:0 - "GET /valor-medio-todas-cidade HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [57]
